In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import sys
sys.path.append("..")

In [147]:
from src.data_generator import *
from src.model import *

In [97]:
X = SemiMartingale(X_0=1.0, vol=1.0)
X.generate(100000, 100)

array([[ 1.        ,  0.88451191,  0.79555051, ...,  0.40638795,
         0.49464316,  0.38552395],
       [ 1.        ,  0.87616902,  0.83434047, ...,  0.27719918,
         0.44049124,  0.37091125],
       [ 1.        ,  0.93897545,  0.84939361, ..., -0.47227529,
        -0.29974246, -0.31806076],
       ...,
       [ 1.        ,  1.06925449,  1.22169392, ...,  2.71338656,
         2.67390504,  2.88465908],
       [ 1.        ,  0.82803863,  0.85826411, ...,  0.9355664 ,
         0.89791103,  0.79736713],
       [ 1.        ,  0.92147093,  0.88659066, ...,  0.91667136,
         1.1083284 ,  1.1113415 ]])

In [98]:
inc = K.constant(X.get_total_increments())
int_var = K.constant(X.integrated_variance())

In [99]:
inc

<tf.Tensor: shape=(100000,), dtype=float32, numpy=
array([-0.614476  , -0.62908876, -1.3180608 , ...,  1.884659  ,
       -0.20263287,  0.1113415 ], dtype=float32)>

In [103]:
p_norm = 2
degree = 10

gaussian_moments = K.constant([gaussian_moment(p) for p in range(1, degree+1)])
z_sample = inc / K.sqrt(int_var)
sample_moments = K.stack([K.mean(z_sample**p, axis=0) for p in range(1, degree+1)])
moment_diffs = K.abs(sample_moments - gaussian_moments)
weights = np.exp(np.linspace(0, -1, num=degree))
K.mean(weights * moment_diffs**p_norm)

<tf.Tensor: shape=(), dtype=float32, numpy=383.74237>

In [101]:
gaussian_moments

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([  0.,   1.,   0.,   3.,   0.,  15.,   0., 105.,   0., 945.],
      dtype=float32)>

In [102]:
sample_moments

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([8.8739133e-04, 9.9823749e-01, 8.6469101e-03, 2.9555957e+00,
       1.2161723e-01, 1.4499716e+01, 1.7317578e+00, 9.8569351e+01,
       2.6073780e+01, 8.4694281e+02], dtype=float32)>

In [136]:
loss1 = MomentLoss(weights = None, degree=10)
loss2 = MomentLoss(weights = 'exponential', degree=10)
loss3 = MomentLoss(weights = 'factorial', degree=10)

In [137]:
print(loss1(inc, int_var))
print(loss2(inc, int_var))
print(loss3(inc, int_var))

tf.Tensor(1033.9674, shape=(), dtype=float32)
tf.Tensor(0.05526517, shape=(), dtype=float32)
tf.Tensor(0.00091985625, shape=(), dtype=float32)


In [171]:
loss = MMDLoss(kernel = 'rbf', length_scale=2.0)

In [172]:
loss.kernel(1.0, 2.0)

<tf.Tensor: shape=(), dtype=float32, numpy=0.7788008>